In [8]:
#| default_exp interface_trimesh

In [9]:
#| export

from blender_tissue_cartography import io as tcio
import numpy as np
import trimesh

In [10]:
from importlib import reload

In [11]:
np.set_printoptions(suppress=True)

## Interfacing with `trimesh`

In this notebook, we define functions to convert our `ObjMesh` class to and from `trimesh`.

Note: `trimesh` represents triangular meshes only, and its way for representing UV information is not ideal. It is not recommended to edit mesh topology in `trimesh` if the UV mapping has already been defined.

In [12]:
reload(tcio)

<module 'blender_tissue_cartography.io' from '/home/nikolas/Documents/UCSB/streichan/numerics/code/python code/jupyter notebooks/blender-tissue-cartography/blender_tissue_cartography/io.py'>

In [13]:
mesh_fname_data = "registration_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj"
mesh_fname_ref = "registration_example/Drosophila_reference_preregistered.obj"

In [14]:
mesh_data = tcio.ObjMesh.read_obj(mesh_fname_data)
mesh_ref = tcio.ObjMesh.read_obj(mesh_fname_ref)

In [15]:
#| export

def convert_to_trimesh(mesh: tcio.ObjMesh, add_texture_info=None) -> trimesh.Trimesh:
    """
    Convert tcio.ObjMesh to trimesh.Trimesh
    
    See https://trimesh.org/trimesh.base.html
    Note: normal information is recalculated. Discards any non-triangle faces.
    
    Texture is saved as a vertex attribute via v_tex_coords_matrix. Note that this discards
    information since a vertex can have multiple texture coordinates!
    For this reason, we also add the texture coordinates as a (n_faces, 3, s)-array attribute
    `face_tex`. Note: this will _not_ be updated if you remesh etc.
    
    Parameters
    ----------
    mesh : tcio.ObjMesh
    add_texture_info : None or bool
        Whether to add texture info to the trimesh.Trimesh. If None, texture is added if
        available for at least one vertex.
    Returns
    -------
    trimesh.Trimesh

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular. discarding non-triangular faces")
    add_texture_info = ((not mesh.only_vertices and len(mesh.texture_vertices) > 0)
                        if add_texture_info is None else add_texture_info)
    if not add_texture_info:
        return trimesh.Trimesh(mesh.vertices, mesh.tris)
    texture = trimesh.visual.texture.TextureVisuals(uv=mesh.vertex_textures)
    converted = trimesh.Trimesh(mesh.vertices, mesh.tris, visual=texture)
    converted.face_tex = tcio.index_else_nan(mesh.texture_vertices, mesh.texture_tris)
    return converted

In [16]:
trimesh_data = convert_to_trimesh(mesh_data)

In [18]:
%%time
trimesh_ref = convert_to_trimesh(mesh_ref)

CPU times: user 354 ms, sys: 7.76 ms, total: 362 ms
Wall time: 362 ms


In [20]:
np.allclose(mesh_ref.vertices, trimesh_ref.vertices)

True

In [21]:
trimesh_normals = trimesh_ref.vertex_normals
trimesh_normals = (trimesh_normals.T / np.linalg.norm(trimesh_normals, axis=-1)).T

np.einsum('vi,vi->v', mesh_ref.vertex_normals, trimesh_normals)

array([0.99904184, 0.99822658, 0.99668032, ..., 0.99417082, 0.99599803,
       0.99721726])

In [23]:
np.allclose(mesh_ref.vertex_textures, trimesh_ref.visual.uv)

True

In [24]:
%%timeit
convert_to_trimesh(mesh_ref)

253 ms ± 5.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
#| export

def convert_from_trimesh(mesh: trimesh.Trimesh, reconstruct_texture_from_faces=True,
                           texture_vertex_decimals=10) -> tcio.ObjMesh:
    """
    Convert trimesh mesh to ObjMesh.
    
    Texture vertices can be reconstructed from face attribute face_tex or from
    vertex attribute vertex_tex_coord_matrix. Reconstruction from face texture can accomodate
    multiple texture coordinates per vertex (e.g. for UV maps with seams).
    
    Texture vertices are rounded to texture_vertex_decimals decimals
    """
    vertices = mesh.vertices
    normals = mesh.vertex_normals
    normals = (normals.T / np.linalg.norm(normals, axis=-1)).T
    if not hasattr(mesh.visual, 'uv'):
        faces = [[3*[v,] for v in f] for f in mesh.faces]
        return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals)
    if hasattr(mesh.visual, 'uv') and not reconstruct_texture_from_faces:
        faces = [[3*[v,] for v in f] for f in mesh.faces]
        return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals, texture_vertices=mesh.visual.uv)
    # reconstruct texture vertices - big pain.
    texture_vertices = np.vstack([mesh.face_tex[:,i,:] for i in [0,1,2]])
    texture_vertices = np.round(texture_vertices, decimals=texture_vertex_decimals)
    texture_vertices_unique, inverse_index = np.unique(texture_vertices, axis=0, return_inverse=True)

    n_faces = mesh.faces.shape[0]
    faces = [[[v, inverse_index[ifc+iv*n_faces], v] for iv, v in enumerate(fc)]
              for ifc, fc in enumerate(mesh.faces)]

    return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals, texture_vertices=texture_vertices_unique)

In [26]:
trimesh_ref = convert_to_trimesh(mesh_ref)

In [28]:
%%time
convert_from_trimesh(trimesh_ref)

CPU times: user 650 ms, sys: 27.6 ms, total: 677 ms
Wall time: 676 ms


In [29]:
mesh_seams = tcio.ObjMesh.read_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv.obj")

In [30]:
trimesh_seams = convert_to_trimesh(mesh_seams,add_texture_info=True)

/home/nikolas/Programs/miniconda3/envs/blender-tissue-cartography/lib/python3.11/site-packages/trimesh/grouping.py:99: RuntimeWarning: invalid value encountered in cast
  stacked = np.column_stack(stacked).round().astype(np.int64)


In [31]:
convert_from_trimesh(trimesh_seams, reconstruct_texture_from_faces=False).texture_vertices.shape

(8159, 2)

In [32]:
mesh_seams_reconverted = convert_from_trimesh(trimesh_seams, reconstruct_texture_from_faces=True)

In [33]:
mesh_seams.texture_vertices.shape, mesh_seams_reconverted.texture_vertices.shape

((8288, 2), (8288, 2))

In [34]:
mesh_seams_reconverted.write_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv_resaved_trimesh.obj")